In [39]:
!pip install datasets transformers accelerate evaluate pandas

# 텍스트 분류 작업에서 모델 파인튜닝
- cola: The Corpus of Linguistic Acceptability의 줄임말로써, '언어학적으로 수용 가능한 문장인지' 판별하는 태스크

In [40]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

In [41]:
task = "cola"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## 데이터셋 로딩

Datasets 라이브러리를 사용하여 데이터를 다운로드하고 평가에 필요한 메트릭을 가져올 것입니다(모델을 벤치마크와 비교하기 위해). 이는 `load_dataset`과 `load_metric` 함수를 사용하여 쉽게 수행할 수 있습니다.

In [42]:
from datasets import load_dataset
from evaluate import load

`mnli-mm`이 특수 코드인 것을 제외하고는 작업 이름을 해당 함수에 직접 전달할 수 있습니다. `load_dataset`은 다음에 이 셀을 실행할 때 다시 다운로드하지 않도록 데이터셋을 캐시합니다.

In [43]:
dataset = load_dataset("glue", task)
metric = load('glue', task)

`dataset` 객체 자체는 [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict)로, 훈련, 검증 및 테스트 세트에 대한 키를 포함합니다

In [44]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

실제 요소에 접근하려면 먼저 분할을 선택한 다음 인덱스를 제공해야 합니다:

In [45]:
dataset["train"][1000]

{'sentence': 'I read three his books.', 'label': 0, 'idx': 1000}

In [46]:
dataset["train"][30]

{'sentence': 'John danced waltzes across the room.', 'label': 1, 'idx': 30}

In [47]:
dataset["train"] = dataset["train"].select(list(range(2000)))
dataset["validation"] = dataset["validation"].select(list(range(50)))
dataset["test"] = dataset["test"].select(list(range(50)))

In [48]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 50
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 50
    })
})

메트릭은 [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric)의 인스턴스입니다:

In [49]:
metric

EvaluationModule(name: "glue", module_type: "metric", features: {'predictions': Value('int64'), 'references': Value('int64')}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = evaluate.load('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(results

예측값과 레이블을 직접 사용하여 `compute` 메서드를 호출할 수 있으며, 메트릭 값이 포함된 딕셔너리를 반환합니다:

In [50]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'matthews_correlation': np.float64(0.0009775171065493646)}

`load_metric`이 작업에 연관된 적절한 메트릭을 로드했다는 점에 주목하세요:

- CoLA: [Matthews Correlation Coefficient](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient)
- MNLI (일치 또는 불일치): 정확도
- MRPC: 정확도 및 [F1 점수](https://en.wikipedia.org/wiki/F1_score)
- QNLI: 정확도
- QQP: 정확도 및 [F1 점수](https://en.wikipedia.org/wiki/F1_score)
- RTE: 정확도
- SST-2: 정확도
- STS-B: [Pearson Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) 및 [Spearman's_Rank_Correlation_Coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)
- WNLI: 정확도

따라서 메트릭 객체는 작업에 필요한 메트릭만 계산합니다.

## 데이터 전처리

이러한 텍스트를 모델에 입력하기 전에 전처리해야 합니다. 이는 Transformers `Tokenizer`에 의해 수행되며, 이름에서 알 수 있듯이 입력을 토큰화하고(사전 훈련된 어휘에서 토큰을 해당 ID로 변환하는 것 포함) 모델이 예상하는 형식으로 변환하며, 모델이 필요로 하는 다른 입력도 생성합니다.

이 모든 작업을 수행하기 위해 `AutoTokenizer.from_pretrained` 메서드로 토크나이저를 인스턴스화합니다. 이는 다음을 보장합니다:

- 사용하려는 모델 아키텍처에 해당하는 토크나이저를 얻습니다
- 이 특정 체크포인트를 사전 훈련할 때 사용된 어휘를 다운로드합니다

해당 어휘는 캐시되므로 다음에 셀을 실행할 때 다시 다운로드되지 않습니다.

In [51]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenizers 라이브러리의 빠른 토크나이저(Rust 기반) 중 하나를 사용하기 위해 위 호출에 `use_fast=True`를 전달합니다. 이러한 빠른 토크나이저는 거의 모든 모델에서 사용할 수 있지만, 이전 호출에서 오류가 발생한 경우 해당 인수를 제거하세요.

이 토크나이저를 한 문장 또는 문장 쌍에 직접 호출할 수 있습니다:

In [52]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

선택한 모델에 따라 위 셀에서 반환된 딕셔너리에서 다른 키들을 볼 수 있습니다. 여기서 하는 작업에는 크게 중요하지 않지만(나중에 인스턴스화할 모델에서 필요하다는 것만 알면 됩니다), 관심이 있다면 [이 튜토리얼](https://huggingface.co/transformers/preprocessing.html)에서 더 자세히 알아볼 수 있습니다.

데이터셋을 전처리하기 위해 문장이 포함된 열의 이름이 필요합니다. 다음 딕셔너리는 작업과 열 이름의 대응 관계를 추적합니다:

현재 데이터셋에서 제대로 작동하는지 다시 한 번 확인할 수 있습니다:

In [53]:
sentence1_key, sentence2_key = "sentence", None

print(f"Sentence: {dataset['train'][20][sentence1_key]}")


Sentence: The professor talked us.


그런 다음 샘플을 전처리할 함수를 작성할 수 있습니다. `truncation=True` 인수와 함께 `tokenizer`에 입력하기만 하면 됩니다. 이렇게 하면 선택한 모델이 처리할 수 있는 길이보다 긴 입력이 모델이 허용하는 최대 길이로 잘립니다.

In [54]:
def preprocess_function(examples):
    return tokenizer(examples[sentence1_key], truncation=True)


이 함수는 하나 또는 여러 예제와 함께 작동합니다. 여러 예제의 경우 토크나이저는 각 키에 대해 리스트의 리스트를 반환합니다:

In [55]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 1998, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 2030, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 1996, 2062, 2057, 2817, 16025, 1010, 1996, 13675, 16103, 2121, 2027, 2131, 1012, 102], [101, 2154, 2011, 2154, 1996, 8866, 2024, 2893, 14163, 8024, 3771, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

데이터셋의 모든 문장(또는 문장 쌍)에 이 함수를 적용하려면 앞서 생성한 `dataset` 객체의 `map` 메서드를 사용하기만 하면 됩니다. 이렇게 하면 `dataset`의 모든 분할에 있는 모든 요소에 함수가 적용되므로 훈련, 검증 및 테스트 데이터가 하나의 명령으로 전처리됩니다.

In [56]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

## 모델 파인튜닝

이제 데이터가 준비되었으므로 사전 훈련된 모델을 다운로드하고 파인튜닝할 수 있습니다. 모든 작업이 문장 분류에 관한 것이므로 `AutoModelForSequenceClassification` 클래스를 사용합니다. 토크나이저와 마찬가지로 `from_pretrained` 메서드가 모델을 다운로드하고 캐시합니다. 문제의 레이블 수만 지정하면 됩니다(회귀 문제인 STS-B와 3개의 레이블이 있는 MNLI를 제외하고는 항상 2개):

In [57]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


`Trainer`를 인스턴스화하려면 두 가지를 더 정의해야 합니다. 가장 중요한 것은 [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments)로, 훈련을 사용자 정의하는 모든 속성을 포함하는 클래스입니다. 모델의 체크포인트를 저장하는 데 사용될 폴더 이름 하나가 필요하며, 다른 모든 인수는 선택사항입니다:

In [63]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
# metric_name = "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-{task}",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    report_to="none"
)

In [64]:
metric_name

'matthews_correlation'

여기서는 각 에포크 끝에 평가를 수행하도록 설정하고, 학습률을 조정하고, 노트북 상단에서 정의한 `batch_size`를 사용하고, 훈련 에포크 수와 가중치 감쇠를 사용자 정의합니다. 최적의 모델이 훈련 끝의 모델이 아닐 수 있으므로 `Trainer`에게 훈련 끝에 저장한 최적의 모델(`metric_name`에 따라)을 로드하도록 요청합니다.



`Trainer`에 대해 정의해야 할 마지막 것은 예측에서 메트릭을 계산하는 방법입니다. 이를 위한 함수를 정의해야 하며, 이 함수는 앞서 로드한 `metric`을 사용하기만 하면 됩니다. 예측된 로짓의 argmax를 취하면 됩니다(STS-B의 경우 마지막 축을 압축하기만 하면 됩니다):

In [65]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis=1)

    return metric.compute(predictions=predictions, references=labels)

그런 다음 이 모든 것을 데이터셋과 함께 `Trainer`에 전달하기만 하면 됩니다:

In [66]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-3132099784.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


이제 `train` 메서드를 호출하기만 하면 모델을 파인튜닝할 수 있습니다:

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


`evaluate` 메서드로 `Trainer`가 최적의 모델을 제대로 다시 로드했는지 확인할 수 있습니다(마지막 모델이 아닌 경우):

In [ ]:
trainer.evaluate()

In [ ]:
# model.save_pretrained("distilbert-base-uncased_glue_trained")